# Example Unbinned Likelihood Fits

In [1]:
import numpy as np
import tensorflow as tf # tf v1.4.0 or later required

import itertools # for fast looping
import time # for timing loop
# from iminuit import Minuit # http://iminuit.readthedocs.io/en/latest/installation.html
import scipy.stats as stats
import scipy.optimize as op

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import pandas as pd

# Show the package versions used
for package in [np, tf]:
    print('{} v{}'.format(package.__name__, package.__version__))

/home/mcf/anaconda3/envs/edwardenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


numpy v1.13.3
tensorflow v1.4.0


Setup a Pandas dataframe for record keeping

In [2]:
fit_df = pd.DataFrame(columns=('Fit Program', 'Number of Trials', 'Number of Events', 'Mean Time'),
                      index=np.arange(0,4))
fit_df_row = 0

First we will define the function that we are going to sample from and then fit to

In [3]:
# This is probably not even close to how I should do things
import math
def my_formula(formula_lambda, *args):
    """
    Create and evaluate a function
    
    Args:
        formula_lambda: `lambda`
        *args: parameters to evaluate the formula with
    
    Returns:
        The formula passed evaluated at the parameters passed
    
    Example:
    >>> my_formula(lambda x, y: x * y, 1, 2)
    2
    >>> my_formula(lambda x, mu, sigma: \
                  math.exp(-1.0*(x - mu)*(x - mu)/math.sqrt(2*math.pi))/(sigma*math.sqrt(2*math.pi)),
                  0, 0, 1)
    0.3989422804014327
    """
    return formula_lambda(*args)

In [4]:
my_formula(lambda x, mu, sigma: \
           math.exp(-1.0*(x - mu)*(x - mu)/math.sqrt(2*math.pi))/(sigma*math.sqrt(2*math.pi)),
           0, 0, 1)

0.3989422804014327

However, let's just use TensorFlow's Normal distribution as it already exists

## Fit in TensorFlow

Kyle Lo has a very nice writeup on this: http://kyleclo.github.io/maximum-likelihood-in-tensorflow-pt-1/

In [2]:
def center_and_scale(x):
    """
    Center and scale input data
    
    Args:
        x: `ndarray` of `floats` or `ints`, the data to be transformed.

    Returns:
        An `ndarray` of `floats` or `ints` that has mean and variance of 1
    """
    center = x.min()
    scale = x.max() - x.min()
    return (x - center) / scale

In [3]:
mu_true = 0.5
sigma_true = 1.5

n_events = 1000000
n_trials = 10

TYPE = tf.float32

In [4]:
# tensor for data
X = tf.placeholder(dtype=TYPE)

# mu_tf = tf.Variable(initial_value=np.random.normal(0., 0.1),
#                  dtype=TYPE)
# sigma_tf = tf.Variable(initial_value=tf.abs(np.random.normal(1., 0.1)),
#                        dtype=TYPE,
#                        constraint=lambda x: tf.abs(x))
mu_tf = tf.Variable(initial_value=tf.random_normal(shape=[], mean=0., stddev=0.1,
                                                dtype=TYPE),
                    name='mu',
                    dtype=TYPE)

sigma_tf = tf.Variable(initial_value=tf.abs(tf.random_normal(shape=[], mean=1., stddev=0.1,
                                                dtype=TYPE)),
                       dtype=TYPE,
                       name='sigma',
                       constraint=lambda x: tf.abs(x))

In [5]:
# loss function
gaussian_dist = tf.distributions.Normal(loc=mu_tf, scale=sigma_tf)
log_prob = gaussian_dist.log_prob(value=X)
negative_log_likelihood = -1.0 * tf.reduce_sum(log_prob)

# optimizer
# optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
# optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(loss=negative_log_likelihood)

# gradient
grad = tf.gradients(negative_log_likelihood, [mu_tf, sigma_tf])

Show that the fit can work at all

In [6]:
def mle_fit(sess,
            parameters,
            true_parameters,
            loss,
            optimizer,
            n_events=1000000,
            iteration_max = 10000,
           parameter_tolerance=1e-6,
           loss_tolerance=1e-6,
           gradient_tolerance=1e-6,
           dtype=tf.float32):
    """
    Args:
        sess: `TensorFlow Session`, the session to run in.
        parameters: `array`, the model parameters.
        true_parameters: `array`, the true parameters.
        loss: `callable`, the function to be minimized.
        optimizer: `tf.train Optimizer`, the optimizer used.
    
    Returns:
        results: `array`, the fitted values of parameters
    """
    
    # define the graph
    train_op = optimizer.minimize(loss=loss)
    
    mu_tf, sigma_tf = parameters
    mu_true, sigma_true = true_parameters
#     x_obs = center_and_scale(np.random.normal(loc=mu_true, scale=sigma_true, size=n_events))
    x_obs = np.random.normal(loc=mu_true, scale=sigma_true, size=n_events)
    
    # redo the obs steps using tensors
#     step_mu = tf.TensorArray(dtype=dtype)
#     step_sigma = tf.TensorArray(dtype=dtype)
    
    # initialize
    sess.run(fetches=tf.global_variables_initializer())
    
    step = 1
    obs_mu, obs_sigma = sess.run(fetches=[[mu_tf], [sigma_tf]])
    obs_loss = sess.run(fetches=[negative_log_likelihood], feed_dict={X: x_obs})
    
    while True:
        # gradient step (variables are updated here)
        sess.run(fetches=train_op, feed_dict={X: x_obs})

        # update parameters
        new_mu, new_sigma = sess.run(fetches=[mu_tf, sigma_tf])
        diff_norm = np.linalg.norm(np.subtract([new_mu, new_sigma],
                                               [obs_mu[-1], obs_sigma[-1]]))
        if diff_norm < parameter_tolerance:
            print('Parameter convergence in {} iterations!'.format(step))
            break

        # update loss
        new_loss = sess.run(fetches=negative_log_likelihood, feed_dict={X: x_obs})
        loss_diff = np.abs(new_loss - obs_loss[-1]) 
#         loss_diff = tf.abs(new_loss - obs_loss[-1]) 
        
#         if sess.run(loss_diff) < loss_tolerance:
        if loss_diff < loss_tolerance:
            print('Loss function convergence in {} iterations!'.format(step))
            break

        # update gradient
#         new_grad = sess.run(fetches=grad, feed_dict={X: x_obs})
#         grad_norm = np.linalg.norm(new_grad)
#         if grad_norm < gradient_tolerance:
#             print('Gradient convergence in {} iterations!'.format(step))
#             break

        obs_mu.append(new_mu)
        obs_sigma.append(new_sigma)
        obs_loss.append(new_loss)
#         obs_grad.append(new_grad)

        if step >= iteration_max:
            print('Max number of iterations reached without convergence.')
            break

        step += 1
        
    results = [new_mu, new_sigma, new_loss]
    return results

In [9]:
with tf.Session() as sess:
    start_time = time.time()
    results = mle_fit(sess = sess,
            parameters = [mu_tf, sigma_tf],
            true_parameters = [mu_true, sigma_true],
            loss = negative_log_likelihood,
            optimizer = tf.train.AdamOptimizer(learning_rate=0.1))
    
    end_time = time.time()
    time_duration = end_time - start_time
    print("\nThe fit time is {} seconds\n".format(time_duration))
    
    print(results)
#     print('Target MLE: [{:.4f}, {:.4f}]'.format(x_obs.mean(), x_obs.std()))
    print('\nTarget MLE: [{:.4f}, {:.4f}]'.format(mu_true, sigma_true))
    print('Fitted MLE: [{:.4f}, {:.4f}]'.format(results[0], results[1]))

Loss function convergence in 94 iterations!

The fit time is 2.5501160621643066 seconds

[0.50076675, 1.5042545, 1824558.6]

Target MLE: [0.5000, 1.5000]
Fitted MLE: [0.5008, 1.5043]


Time the fit (this should be done in a cleaner way ... this also takes a long time right now)

In [10]:
def sfill(x, max_chars=10, justify='>'):
    """Fill a string with empty characters"""
    return '{}' \
        .format('{:' + justify + str(max_chars) + '}') \
        .format(x)


def sfloat(x, num_chars=10):
    """Stringify a float to have exactly some number of characters"""
    x = float(x)
    num_chars = int(num_chars)
    start, end = str(x).split('.')
    start_chars = len(str(float(start)))
    if start_chars > num_chars:
        raise Exception('Try num_chars = {}'.format(start_chars))
    return '{}' \
        .format('{:' + str(num_chars) + '.' +
                str(num_chars - start_chars + 1) + 'f}') \
        .format(x)

        
TOL_PARAM, TOL_LOSS, TOL_GRAD = 1e-6, 1e-6, 1e-6
MAX_ITER = 10000

start_time = time.time()

with tf.Session() as sess:
    
    for _ in itertools.repeat(None, n_trials):
        # initialize
        sess.run(fetches=tf.global_variables_initializer())
        
        x_obs = center_and_scale(np.random.normal(loc=mu_true, scale=sigma_true, size=n_events))

        step = 1
        obs_mu, obs_phi, obs_sigma = sess.run(fetches=[[mu_tf], [phi_tf], [sigma_tf]])
        obs_loss = sess.run(fetches=[negative_log_likelihood], feed_dict={X: x_obs})
        obs_grad = sess.run(fetches=[grad], feed_dict={X: x_obs})

        while True:
            # gradient step
            sess.run(fetches=train_op, feed_dict={X: x_obs})

            # update parameters
            new_mu, new_phi, new_sigma = sess.run(fetches=[mu_tf, phi_tf, sigma_tf])
            diff_norm = np.linalg.norm(np.subtract([new_mu, new_phi],
                                                   [obs_mu[-1], obs_phi[-1]]))

            # update loss
            new_loss = sess.run(fetches=negative_log_likelihood, feed_dict={X: x_obs})
            loss_diff = np.abs(new_loss - obs_loss[-1])        

            # update gradient
            new_grad = sess.run(fetches=grad, feed_dict={X: x_obs})
            grad_norm = np.linalg.norm(new_grad)

            obs_mu.append(new_mu)
            obs_phi.append(new_phi)
            obs_sigma.append(new_sigma)
            obs_loss.append(new_loss)
            obs_grad.append(new_grad)

            if diff_norm < TOL_PARAM:
                print('Parameter convergence in {} iterations!'.format(step))
                break

            if loss_diff < TOL_LOSS:
                print('Loss function convergence in {} iterations!'.format(step))
                break

            if grad_norm < TOL_GRAD:
                print('Gradient convergence in {} iterations!'.format(step))
                break

            if step >= MAX_ITER:
                print('Max number of iterations reached without convergence.')
                break
                
            #         if (step - 1) % 100 == 0:
#             print(' {} | {} | {} | {} | {}'
#                   .format(sfill(step, len(str(MAX_ITER))),
#                           sfloat(new_mu, MAX_CHARS),
#                           sfloat(new_sigma, MAX_CHARS),
#                           sfloat(new_loss, MAX_CHARS),
#                           sfloat(grad_norm, MAX_CHARS)))

            step += 1
            
            
    
end_time = time.time()
time_duration = end_time - start_time
mean_fit_time = time_duration/n_trials

print("\ntarget mu = {:.4f}, target sigma = {:.4f}".format(x_obs.mean(), x_obs.std()))
print("MLE mu = {:.4f}, MLE sigma = {:.4f}".format(obs_mu[-1], obs_sigma[-1]))

print("\nfit {} points {} times in {} seconds".format(n_events, n_trials, time_duration))
print("The average fit time is {} seconds".format(mean_fit_time))

Loss function convergence in 708 iterations!
Loss function convergence in 701 iterations!
Loss function convergence in 709 iterations!
Loss function convergence in 701 iterations!
Loss function convergence in 707 iterations!
Parameter convergence in 714 iterations!
Loss function convergence in 710 iterations!
Loss function convergence in 674 iterations!
Loss function convergence in 666 iterations!
Parameter convergence in 705 iterations!

target mu = 0.5147, target sigma = 0.0982
MLE mu = 0.5147, MLE sigma = 0.0982

fit 1000000 points 10 times in 278.739785194397 seconds
The average fit time is 27.873978519439696 seconds


In [11]:
fit_df.loc[fit_df_row] = ['TensorFlow', n_trials, n_events, mean_fit_time]
fit_df_row += 1

--- This is old and bad and should be cleaned up

In [12]:
def sample_model(model, n_samples, TYPE=np.float32):
    """
    Sample the model n_samples times
    
    Args:
        model: `tf.distributions` The model
        n_samples: `int` The number of times the model is samples
    
    Returns:
        The sampled points and their values: x,y
        x: model.sample(n_samples)
        y: model.prob(x)
    """
    x = model.sample(n_samples)
    y = model.prob(x)
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        return sess.run(x), sess.run(y)

In [13]:
def normal_log(X, mu, sigma, TYPE=np.float32):
    """The log of Normal(X | mu, sigma)"""
    return -tf.log(tf.constant(np.sqrt(2 * np.pi), dtype=TYPE) * sigma) - \
        tf.pow(X - mu, 2) / (tf.constant(2, dtype=TYPE) * tf.pow(sigma, 2))

In [14]:
def nll(X, mu, sigma, TYPE=np.float32):
    """The NLL of Normal(X | mu, sigma)"""
    return -tf.reduce_sum(normal_log(X, mu, sigma, TYPE))

c.f. https://gist.github.com/ibab/45c3d886c182a1ea26d5

In [15]:
# # pdf of Gaussian of variable x with mean mu and standard deviation sigma
# # mu = tf.Variable(0.)
# mu = tf.Variable(np.float64(0.))
# # sigma = tf.Variable(1.)
# sigma = tf.Variable(np.float64(1.))
# model_tf = tf.distributions.Normal(loc=mu, scale=sigma)

In [16]:
# # My memory on my computer seems to fill up and stay full very fast by rerunning this
# # so it seems that Python is not releasing anything

# # MLE attempt
# TYPE = np.float64 # This is required(?) for the fit to converge
# # TYPE = np.float32

# n_events = 1000000 # time of fit is very dependent on n_events
# n_trials = 10

# sess = tf.Session()

# # def func(mu_, sigma_):
# #     return sess.run(nll_, feed_dict={mu: mu_, sigma: sigma_})

# def func(x):
#     return sess.run(nll_, feed_dict={mu: x[0], sigma: x[1]})

# mu_true = tf.Variable(np.float64(0.5))
# sigma_true = tf.Variable(np.float64(1.5))
# model_tf = tf.distributions.Normal(loc=mu_true, scale=sigma_true)

# start_time = time.time()
# for _ in itertools.repeat(None, n_trials):
#     data = np.random.normal(0.5, 1.5, n_events)
# #     data, _ = sample_model(model_tf, n_events) # this is wrong for some reason
    
#     # Define data as a variable so that it will be cached
#     X = tf.Variable(data, name='data')
    
#     mu = tf.Variable(TYPE(1), name='mu')
#     sigma = tf.Variable(TYPE(1), name='sigma')
    
#     init = tf.global_variables_initializer()
#     sess.run(init)
    
#     nll_ = nll(X, mu, sigma, TYPE)
    
#     # To guard against excessive output
#     if n_trials > 1:
#         print_level = 0
#     else:
#         print_level = 1
    
#     ret = op.minimize(func, x0=[10, 10], bounds=[(-1, 100), (0.00001, 100)])
# #     print(ret.x, ret.fun) # x is an array of fit values, fun is the value of the function passed
    
# end_time = time.time()
# time_duration = end_time - start_time
# mean_fit_time = time_duration/n_trials

# print("true mu = {}, true sigma = {}".format(sess.run(mu_true), sess.run(sigma_true)))
# print("mu = {}, sigma = {}".format(ret.x[0], ret.x[1]))
      
# sess.close()

# print("\nfit {} points {} times in {} seconds".format(n_events, n_trials, time_duration))
# print("The average fit time is {} seconds".format(mean_fit_time))

In [17]:
# print(data)

In [18]:
# x = np.linspace(-5.0, 5.0, num=10000)
# plt.figure(1)

# sample_distribution = stats.norm.pdf(x, ret.x[0], ret.x[1])

# # Plot histogram of samples
# hist_count, bins, _ = plt.hist(data, 50, normed=True) #Norm to keep distribution in view
# plt.plot(x, sample_distribution, linewidth=2, color='black')

# plt.show()

---

In [19]:
# # Example
# test_x, test_y = sample_model(model_tf, 50)

In [20]:
# print(test_x)
# print(test_y)

---

## Fit in SciPy

### Fit using `rv_continuous.fit()`

In [21]:
TYPE = np.float64

# Using the continuous random variable class
# https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.rv_continuous.html
from scipy.stats import rv_continuous
class model_gen(rv_continuous):
    """Normal distribution"""
    def _pdf(self, x):
        return np.exp(-x**2 / 2.) / np.sqrt(2.0 * np.pi)

Show that the fit can work at all

In [22]:
scipy_gaussian = model_gen(name='gaussian')

mu_true = 0.5
sigma_true = 1.5

X = np.random.normal(mu_true, sigma_true, n_events).astype(TYPE)

# Return MLEs for shape, location, and scale parameters from data
# https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.rv_continuous.fit.html
scipy_ret = scipy_gaussian.fit(X)

print('MLE mean: {}'.format(scipy_ret[0]))
print('MLE standard deviation: {}'.format(scipy_ret[1]))

MLE mean: 0.5016512908960371
MLE standard deviation: 1.500075735274


Time the fit

In [23]:
start_time = time.time()

for _ in itertools.repeat(None, n_trials):
    X = np.random.normal(mu_true, sigma_true, n_events).astype(TYPE)
    scipy_ret = scipy_gaussian.fit(X)
    
end_time = time.time()
time_duration = end_time - start_time
mean_fit_time = time_duration/n_trials

print("true mu = {}, true sigma = {}".format(mu_true, sigma_true))
print("MLE mu = {}, MLE sigma = {}".format(scipy_ret[0], scipy_ret[1]))

print("\nfit {} points {} times in {} seconds".format(n_events, n_trials, time_duration))
print("The average fit time is {} seconds".format(mean_fit_time))

true mu = 0.5, true sigma = 1.5
MLE mu = 0.5006687689956756, MLE sigma = 1.4992090511031169

fit 1000000 points 10 times in 207.31614136695862 seconds
The average fit time is 20.73161413669586 seconds


In [24]:
fit_df.loc[fit_df_row] = ['SciPy: rv_continuous', n_trials, n_events, mean_fit_time]
fit_df_row += 1

### Fit using `optimize.minimize()`

Define the NLL for our Gaussian model --- the function we want to minimize

In [25]:
def nll(params, data):
    """
    The negative log likelihood of Normal(X | loc, scale).
    Designed to be a callable for scipy.optimize.minimize()
    
    Args:
        params: `tuple` or `array` of `floats` or `ints`, the parameters of the model.
        data: `ndarray` of `floats` or `ints`, the sampeled data.

    Returns:
        `float`, the negative log likelihood of Normal(data | loc, params).
        
    Example:
    >>> loc, scale, n_events = 0, 1, 1000
    >>> X = np.random.normal(loc, scale, n_events)
    >>> nll([loc, scale], X)
    502.01978041191808
    """
    loc, scale = params
    return len(data) * np.log(scale) + np.sum((data - loc) ** 2) / (2 * scale ** 2)

Show that the fit can work at all

In [35]:
mu_true = 0.5
sigma_true = 1.5

X = np.random.normal(mu_true, sigma_true, n_events).astype(TYPE)

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
op_ret = op.minimize(nll, x0=[10,10], args=(X), bounds=((None, None), (0.00001, None)),
                    tol=1e-05)

print('MLE mean: {}'.format(op_ret.x[0]))
print('MLE standard deviation: {}'.format(op_ret.x[1]))

MLE mean: 0.49685686919191
MLE standard deviation: 1.4990275269080338


Time the fit

In [27]:
start_time = time.time()

for _ in itertools.repeat(None, n_trials):
    X = np.random.normal(mu_true, sigma_true, n_events).astype(TYPE)
    op_ret = op.minimize(nll, x0=[10,10], args=(X), bounds=((None, None), (0.00001, None)))
    
end_time = time.time()
time_duration = end_time - start_time
mean_fit_time = time_duration/n_trials

print("true mu = {}, true sigma = {}".format(mu_true, sigma_true))
print("MLE mu = {}, MLE sigma = {}".format(op_ret.x[0], op_ret.x[1]))

print("\nfit {} points {} times in {} seconds".format(n_events, n_trials, time_duration))
print("The average fit time is {} seconds".format(mean_fit_time))

true mu = 0.5, true sigma = 1.5
MLE mu = 0.49725592819625203, MLE sigma = 1.500443546344019

fit 1000000 points 10 times in 5.191132545471191 seconds
The average fit time is 0.5191132545471191 seconds


In [28]:
fit_df.loc[fit_df_row] = ['SciPy: optimize', n_trials, n_events, mean_fit_time]
fit_df_row += 1

---

## Fit in RooFit

In [29]:
fit_df.loc[fit_df_row] = ['RooFit', n_trials, n_events, mean_fit_time]
fit_df_row += 1

## Fit Results Summary

In [31]:
fit_df

,Fit Program,Number of Trials,Number of Events,Mean Time
0,TensorFlow,10,1000000,27.874
1,SciPy: rv_continuous,10,1000000,20.7316
2,SciPy: optimize,10,1000000,0.519113
3,RooFit,10,1000000,0.519113
